In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools
import random
import requests
import os
import json
import time
import psutil

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

In [2]:
import requests

index_url = 'https://api.github.com/repos/numenta/NAB/contents/data'

# Fetching file names from the index URL
response = requests.get(index_url)

# Check if the response was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    index_data = response.json()
    
    # Extract directory names
    directories = [file['name'] for file in index_data if file.get('type') == "dir"]
    print(directories)
else:
    print("Failed to fetch data:", response.status_code)

Failed to fetch data: 403


In [3]:
base_url = 'https://raw.githubusercontent.com/numenta/NAB/master/data/'
data = {}

def addFolderAndReadAll(d_name):
    data[d_name] = {}
    response = requests.get(index_url + '/' + d_name)
    index_data = response.json()

    csv_files = [ file['name'] for file in index_data if file['type'] == "file"]
    csvs_num = 0
    for f_name in csv_files:
        data[d_name][f_name] = pd.read_csv(base_url + d_name + '/' + f_name)
        csvs_num += 1
    return csvs_num

csvs_num = sum([addFolderAndReadAll(d_name) for d_name in directories])

NameError: name 'directories' is not defined

In [ ]:
# Function to get a random start date from the DataFrame index
def get_random_start_date(index):
    return np.random.choice(index)

# Main function to repeat the process until non-None frequency is obtained
def find_non_none_frequency(df, offset=9):
    while True:
        # Get a random start date from the DataFrame index
        start_date = pd.to_datetime(get_random_start_date(df.index))

        # Find the index of the end date by moving 9 steps through the indices
        end_date_index = df.index.get_loc(start_date) + offset

        # Check if the end date index is within the range of the DataFrame index
        if end_date_index < len(df.index):
            # Calculate the end date using the index
            end_date = df.index[end_date_index]

            # Infer frequency within the specified date range
            subset_df = df.loc[start_date:end_date]
            freq = pd.infer_freq(subset_df.index)

            if freq is not None:
                print("Inferred frequency within range", start_date, "-", end_date, ":", freq)
                return freq  # Exit the loop and return the inferred frequency

In [ ]:
def max_consecutive_missing_dates(inferred_freq, missing_dates):
    # Function to check if two dates are consecutive based on the inferred frequency
    def are_consecutive(date1, date2, freq):
        # Calculate the difference between dates based on the inferred frequency
        diff = date2 - date1
        # Check if the difference matches the frequency
        if freq == 'D':
            return diff.days == 1
        elif freq.endswith('H')| freq.endswith('h'):
             # If the frequency ends with 'H', check if it represents hourly intervals
            if freq[:-1]:  # Check if there is a multiplier
                  interval = int(freq[:-1])
                  return diff.total_seconds() == interval * 3600
            else:
                   # If no multiplier is provided, it's assumed to be one hour
                   return diff.total_seconds() == 3600
        elif freq.endswith('T') | freq.endswith('min') :
            if freq.endswith('T'):
                # Extract the interval from the frequency string
                interval = int(freq[:-1])
                return diff.seconds // 60 == interval
            else:
                interval = int(freq[:-3])
                return diff.seconds // 60 == interval
        else:
            raise ValueError("Unsupported frequency: {}".format(freq))

    # Initialize variables to track maximum length and current length
    max_consecutive_missing = 0
    current_consecutive_missing = 0

    # Iterate over the missing dates
    for i in range(1, len(missing_dates)):
        # Check if the current date is consecutive with the previous date
        if are_consecutive(missing_dates[i - 1], missing_dates[i], inferred_freq):
            # Increment current consecutive missing count
            current_consecutive_missing += 1
        else:
            # Update maximum consecutive missing count if needed
            max_consecutive_missing = max(max_consecutive_missing, current_consecutive_missing)
            # Reset current consecutive missing count
            current_consecutive_missing = 0

    # Update max_consecutive_missing if current_consecutive_missing is still greater
    max_consecutive_missing = max(max_consecutive_missing, current_consecutive_missing)

    return max_consecutive_missing

: 

In [ ]:
def preprocess(df, f_name):
    # Convert 'timestamp' column to datetime format and rename it to 'ds'
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Removing the duplicate rows
    df = df[~df.duplicated(keep='first')]

    duplicated_dates_length = len(df[df['timestamp'].duplicated(keep=False)])

    if  duplicated_dates_length > 0:
      print("Number of Duplicated Dates in "+ f_name + ": "+ str(duplicated_dates_length))
      # To make the mean as the value for the numerical columns if there are different values for a particular date
      df = df.groupby('timestamp').mean()
      # Reset index to bring 'timestamp' column back
      df.reset_index(inplace=True)

    df.set_index(['timestamp'], inplace=True)
    df.sort_index()

    # Create a date range with hourly frequency covering the entire time range
    start_date = df.index.min()
    end_date = df.index.max()

    #inferred_freq = pd.infer_freq(df.index)
    inferred_freq = find_non_none_frequency(df)

    if inferred_freq is None:
      inferred_freq = default_freq # setting the default frequency
      print("Cannot infer the frequency of the timestamp of the dataset "+ f_name+ " .Therefore the default frequency of " + default_freq+ " will be used")

    expected_date_range = pd.date_range(start=start_date, end=end_date, freq=inferred_freq)

    # Find the missing date entries
    missing_dates = expected_date_range[~expected_date_range.isin(df.index)]
    # Print or work with the list of missing dates
    print("Number of Missing Dates in "+ f_name + ": "+ str(len(missing_dates))+"\n")

    if len(missing_dates) > 0:
      df = df.asfreq(inferred_freq)
      df.sort_index()

      # Call the function with inferred_freq and missing_dates parameters
      max_consecutive = max_consecutive_missing_dates(inferred_freq, missing_dates)
      print("Maximum length of consecutive missing dates:", max_consecutive)
      if max_consecutive > 3:
        print("It is better to use other imputation method rather than linear interpolation")

      df['value'] = df['value'].interpolate(method='linear')

    return df

: 

In [ ]:
url = 'https://raw.githubusercontent.com/numenta/NAB/master/labels/combined_labels.json'

response = requests.get(url)

if response.status_code == 200:
    labels = json.loads(response.text)
else:
    print("Failed to retrieve data from the URL:", response.status_code)

: 

In [ ]:
# List of directories
dirs = ['realAdExchange', 'realAWSCloudwatch', 'realKnownCause', 'realTweets', 'artificialWithAnomaly', 'artificialNoAnomaly']
#dirs = ['realAdExchange']
#dirs = ['artificialNoAnomaly']

# Loop through each directory
for dir in dirs:
    for f_name in data[dir]:
        print("")
        print(f"Iterating over file: {dir} / {f_name}")
        df = preprocess(data[dir][f_name], f_name)
        labels_of_one_file = labels[dir+'/'+f_name]
        df['is_anomaly'] = 0
        for anomalous_timestamp in labels_of_one_file:
            anomalous_timestamp = pd.to_datetime(anomalous_timestamp)
            try:
                df.at[anomalous_timestamp, 'is_anomaly'] = 1  # Set is_anomaly to 1 at the index location
            except KeyError:
                print(f"Anomalous timestamp {anomalous_timestamp} not found in data[{dir}][{f_name}].")
                pass
        data[dir][f_name] = df  # Assign the modified DataFrame back to the data dictionary

: 

In [ ]:
# Specify the directory and file name
dir = 'realAWSCloudwatch'
f_name = 'ec2_cpu_utilization_ac20cd.csv'


# Retrieve the DataFrame
df = data[dir][f_name]

: 

In [ ]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-04-15')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df


: 

In [ ]:
f_name = 'ec2_cpu_utilization_5f5533.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

: 

In [ ]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-02-25')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

: 

In [ ]:
f_name = 'grok_asg_anomaly.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

: 

In [ ]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-01-29')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

: 

In [ ]:
f_name = 'rds_cpu_utilization_cc0c53.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

: 

In [ ]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-02-25')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

: 

In [ ]:
def split_data(df, train_ratio=0.6):
    train_size = int(len(df) * train_ratio)
    train, val = df[:train_size], df[train_size:]
    return train, val

: 

In [ ]:
# Create an empty DataFrame with the desired columns
original_values_df = pd.DataFrame(columns=['dir', 'values'])

for dir in dirs:
    print(f"Iterating over directory: {dir}")

    # Iterate over each file in the current directory
    for file_name in data[dir]:
        df = data[dir][file_name]
        # Assuming df is defined somewhere in the loop
        train, val = split_data(df)

        # Create a new row with the directory and values as a list
        new_row = pd.DataFrame({'dir': [dir], 'file_name':[file_name], 'values': [val['value'].tolist()]})

        # Concatenate the new row to the original_values_df
        original_values_df = pd.concat([original_values_df, new_row], ignore_index=True)


original_values_df

: 

Loading of datasets (time series extracted features)

In [ ]:
df = pd.read_csv('../dataset_preparation/ranking_dataset.csv')
df.head() 

: 

In [ ]:
predicted_df_final = pd.read_csv('../dataset_preparation/predicted_results.csv')
predicted_df_final.head()

: 

In [ ]:
#original_values_df = pd.read_csv('../dataset_preparation/original_values.csv')
#original_values_df.head()

: 

In [ ]:
mae_df_final = pd.read_csv('../dataset_preparation/mae_results.csv')
mae_df_final.head()

: 

In [ ]:
mape_df_final = pd.read_csv('../dataset_preparation/mape_results.csv')
mape_df_final.head()

: 

Ranking of base models based on the evaluation metrics

In [ ]:
# Define a function to rank models based on MAE values for each row
def rank_models(row):
    mae_values = row[['exponential_smoothing','xgboost', 'random_forest', 'vae']] # 'lstm' ,'gru',
    #mae_values = row[['exponential_smoothing', 'xgboost', 'random_forest']]
    model_rank = mae_values.sort_values().index.tolist()
    return model_rank

: 

In [ ]:
# Apply the function to each row of the DataFrame
df['model_rank'] = df.apply(rank_models, axis=1)
df.head()

: 

In [ ]:
df

: 

Stacking Appraoch to combine inputs from base model - RandomForest Regressor 

In [ ]:
def stacked_model_predictions(val, base_preds):
     # Convert string inputs to lists of floats
     # Convert string inputs to lists of floats for each column in base_preds
    base_preds = [list(map(float, col.strip('[]').split(','))) for col in base_preds]
    
    # Convert lists to numpy arrays and transpose to get the correct shape
    base_preds = np.asarray(base_preds).T

    print("Type of base_preds:", type(base_preds))
    print("Shape of base_preds:", base_preds.shape)
    #val = list(map(float, val[0].strip('[]').split(',')))
    
    
    
    # Convert lists to numpy arrays
    val = np.asarray(val).reshape(-1)
    #base_preds = np.asarray(base_preds).reshape(-1, 1)
    # Check the type and shape after conversion
    print("Type of val:", type(val))
    print("Shape of val:", val.shape)
    

    # Check if base_preds and val have the same length
    if base_preds.shape[0] != len(val):
        raise ValueError("base_preds and val must have the same length.")
    # Ensure base_preds and val are numpy arrays

    
    # Check if base_preds and val have the same length
    if len(base_preds) != len(val):
        raise ValueError("base_preds and val must have the same length.")
    # Splitting features and target variable sequentially
    train_size = int(0.65 * len(val))  # Assuming the split ratio is 80-20
    X_train, y_train = base_preds[:train_size], val[:train_size]
    X_val, y_val = base_preds[train_size:], val[train_size:]

    # Define parameter grid for Random Forest
    param_grid = {
        'n_estimators': [25, 50, 100, 150, 200],  # Number of trees in the forest
        'max_depth': [None, 10, 20, 30],      # Maximum depth of the tree
        'min_samples_split': [2, 5, 8, 10, 15],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4, 6]     # Minimum number of samples required to be at a leaf node
    }

    # Initialize Random Forest regressor
    rf = RandomForestRegressor(random_state=42)

    search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=2, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)

    print("Stacking Approach")

    # Print the best estimator found
    print(search.best_estimator_)

    # Make predictions using the best model
    y_pred = search.best_estimator_.predict(X_val)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_val, y_pred)
    print("Mean Absolute Error (MAE):", mae)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error (MSE):", mse)

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Calculate Mean Absolute Percentage Error (MAPE)
    mape = mean_absolute_percentage_error(y_val, y_pred)
    print("Mean Absolute Percentage Error (MAPE):", mape)

    print("")

    return y_pred, y_val, mae, mse, rmse, mape


: 

In [ ]:
# Create an empty column named 'ensemble_size'
df['ensemble_size'] = np.nan
# Display the DataFrame with the new empty column
df.head(2)

: 

Logic to find the appropriate ensemble size for training

In [ ]:
def find_ensemble_size(model_rank, unique_id, predicted_df_final, mae_df_final, original_values_df):

    threshold_mape=0.05
    # Extract directory and file name from unique_id
    dir_name, file_name = unique_id.split('/')

    # Get the first element from the list of model_rank
    model_name = model_rank[0]

    # Find the row in mae_df dataframe that matches the directory and file name
    row = mae_df_final[(mae_df_final['dir'] == dir_name) & (mae_df_final['file_name'] == file_name)]

    # Find the value in the column that matches the model_name
    model_mae = row[model_name].iloc[0]

    # Determine the ensemble size based on the model MAE value
    if model_mae < threshold_mape:
        print(f"No need for stacking approach, since the first model {model_name} has MAE value {model_mae} less than the threshold value {threshold_mape}")
        return 1
    else:
        previous_mae = model_mae
        print("Going for stacking Approach")
        i = 1  # Initialize the count of models
        while i < len(model_rank):
            print("Going for the next iteration ", i+1, " of the same loop")
            threshold_mape=threshold_mape+0.05
            i += 1  # Increment the count of models
            models_to_use = model_rank[:i]  # Take the first i models from the model_rank list
            base_preds = []  # Initialize base_preds as a list
            # Get the predicted values for the selected models
            for model in models_to_use:
                # Find the respective row in predicted_df
                model_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
                # Get the predicted value for the model
                pred_value = model_row[model].iloc[0]

                print(pred_value)
                # Append the predicted value to base_preds
                base_preds.append(pred_value)

            val_row = original_values_df[(original_values_df['dir'] == dir_name) & (original_values_df['file_name'] == file_name)]
            val = val_row['values'].iloc[0]

            base_preds = np.stack(base_preds, axis=-1)
            y_pred, y_val, mae, mse, rmse, mape = stacked_model_predictions(val, base_preds)

            if mae > previous_mae:
                print("MAE increased after adding", i, "models, from", previous_mae, "to", mae, "so returning the previous ensemble size")
                return i - 1

            # Update previous MAE with current MAE
            previous_mae = mae

            # If MAE is less than 0.05, return the current ensemble size
            if mae < threshold_mape:
                return i        

    # If none of the models have MAE less than 0.05, return the total count of models
    return len(model_rank)

: 

In [ ]:
for index, row in df.iterrows():
    # Extract values from the current row
    model_rank = row['model_rank']
    unique_id = row['unique_id']

    # Print the unique ID before calling the function
    print("Unique ID:", unique_id)

    # Call the function to find ensemble size
    ensemble_size = find_ensemble_size(model_rank, unique_id, predicted_df_final, mae_df_final, original_values_df)

    print("")

    # Assign the ensemble size to the 'ensemble_size' column
    df.at[index, 'ensemble_size'] = ensemble_size

: 

In [ ]:
# Display the updated DataFrame
df

: 

In [ ]:
# Assuming df_features is your DataFrame

# Convert 'ensemble_size' column from float to integer
df['ensemble_size'] = df['ensemble_size'].astype(int)

# Print the DataFrame to verify the changes
df.head(5)

: 

In [ ]:
# Delete the row with index 3
df = df.drop(index=3)

: 

In [ ]:


# Save the DataFrame
df.to_csv('../dataset_preparation/df_features_with_ensemble_size_2.csv', index=False)  # for CSV file

print("CSV saved successfully")

: 